###BART

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

def rec_cleanning(Text):
    Text = re.sub('http\S+\s*',' ',Text)
    Text = re.sub('RT|cc',' ',Text)
    Text = re.sub('#\S+',' ',Text)
    Text = re.sub('@\S+',' ',Text)
    Text = re.sub('[%s]' % re.escape("""!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"""),' ',Text)
    Text = re.sub(r'[^\x00-\x7f]',' ',Text)
    Text = re.sub('\s+',' ',Text)
    Text = Text.lower()
    return Text


def remove_stopwords(text):
    stop_words= set(stopwords.words("english"))
    word_tokens=word_tokenize(text)
    new_text= [word for word in word_tokens if word not in stopwords]
    return new_text


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
!pip install ohmeow-blurr -q
!pip install bert-score -q

In [ ]:
import pandas as pd
from fastai.text.all import *
from transformers import *
from blurr.data.all import *
from blurr.modeling.all import *

#Get data
df = pd.read_csv('/content/drive/MyDrive/Summary-dataset - db1.csv')
#df = df.dropna().reset_index()

#Select part of data we want to keep
df = df[['Text','Summary']]

#Clean text
df['Text'] = df['Text'].apply(lambda x: x.replace('\n',' '))
df['Summary'] = df['Summary'].apply(lambda x: x.replace('\n',' '))

df['Text'] = df['Text'].apply(lambda x: rec_cleanning(x))
df['Summary'] = df['Summary'].apply(lambda x: rec_cleanning(x))


#df['Text'] = df['Text'].apply(lambda x: remove_stopwords(x))
#df['Summary'] = df['Summary'].apply(lambda x: remove_stopwords(x))

#Select only part of it (makes testing faster)
articles = df.head(100)
articles.head(2)


,Text,Summary
0,core qualifications drafting space planning interiorarchitectural detailing colorand materialapplication proficient with furnituresystems concept development proficient inautocad graphic presentations productspecifications skills 3d approach art agency autocad basic budgets budget color concept concept development content council client clients direction documentation drafting fast floor plans graphic hvac instructor interior design interior design lighting a ess window plumbing presentations design process space planning specification teaching vision written a omplishments lightingdesigne...,interior designer with a distinct understanding ofspatialtheorywho harborsan exceptional design instinctand is uniquely attuned to theimportance ofmateriality lighting composition and the overall design process
1,interior designer with a distinct understanding ofspatialtheorywho harborsan exceptional design instinctand is uniquely attuned to theimportance ofmateriality lighting composition and the overall design process,passionate personwith two years ofexperienceasa marketing coordinator strong adaptability to newworking environmentand willing to learn newskillsand take on newchallenges skilled in designing campaignmaterialsand promotionalitems using indesign and illustrator seeking a position inmarketingwith acompany on along termbasis who is looking fora hardworking goal oriented teamplayer


In [ ]:
#echarlaix/bart-base-cnn-r2-19.4-d35-hybrid

#Import the pretrained model
pretrained_model_name = "facebook/bart-large-cnn"
hf_arch, hf_config, hf_tokenizer, hf_model = BLURR.get_hf_objects(pretrained_model_name, 
                                                                  model_cls=BartForConditionalGeneration)

#Create mini-batch and define parameters
hf_batch_tfm = HF_Seq2SeqBeforeBatchTransform(hf_arch, hf_config, hf_tokenizer, hf_model, 
    task='summarization',
    text_gen_kwargs=
 {'max_length': 248,'min_length': 56,'do_sample': False, 'early_stopping': True, 'num_beams': 4, 'temperature': 1.0, 
  'top_k': 50, 'top_p': 1.0, 'repetition_penalty': 1.0, 'bad_words_ids': None, 'bos_token_id': 0, 'pad_token_id': 1,
 'eos_token_id': 2, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'encoder_no_repeat_ngram_size': 0,
 'num_return_sequences': 1, 'decoder_start_token_id': 2, 'use_cache': True, 'num_beam_groups': 1,
 'diversity_penalty': 0.0, 'output_attentions': False, 'output_hidden_states': False, 'output_scores': False,
 'return_dict_in_generate': False, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2, 'remove_invalid_values': False})


#Prepare data for training
blocks = (HF_Seq2SeqBlock(before_batch_tfm=hf_batch_tfm), noop)
dblock = DataBlock(blocks=blocks, get_x=ColReader('Text'), get_y=ColReader('Summary'), splitter=RandomSplitter())
dls = dblock.dataloaders(articles, batch_size = )

In [ ]:
#Define performance metrics
seq2seq_metrics = {
        'rouge': {
            'compute_kwargs': { 'rouge_types': ["rouge1", "rouge2", "rougeL"], 'use_stemmer': True },
            'returns': ["rouge1", "rouge2", "rougeL"]
        },
        'bertscore': {
            'compute_kwargs': { 'lang': 'fr' },
            'returns': ["precision", "recall", "f1"]}}

#Model
model = HF_BaseModelWrapper(hf_model)
learn_cbs = [HF_BaseModelCallback]
fit_cbs = [HF_Seq2SeqMetricsCallback(custom_metrics=seq2seq_metrics)]

#Specify training
learn = Learner(dls, model,
                opt_func=ranger,loss_func=CrossEntropyLossFlat(),
                cbs=learn_cbs,splitter=partial(seq2seq_splitter, arch=hf_arch)).to_fp16()

#Create optimizer with default hyper-parameters
learn.create_opt() 
learn.freeze()

#Training
learn.fit_one_cycle(3, lr_max=3e-5, cbs=fit_cbs)

TypeError: ignored

In [ ]:
learn.export("/content/drive/MyDrive/Colab Notebooks/Models/BART-summary.")

In [ ]:
for i in range (int(len(df['Text'])/10)):
  outputs = learn.blurr_generate(df['Text'][i], early_stopping=False, num_return_sequences=1)

  for idx, o in enumerate(outputs):
      print(f'=== Prediction {idx+1} ===\n{o}\n')

In [ ]:
learn = load_learner(os.path.join(temp_path, 'adult_sample_model.pk1*))